In [3]:
import pandas as pd
import numpy as np
import pickle
import collections
#Euclidean distance of two poin ts a and b. Use lat and long. 
#dist = numpy.linalg.norm(a-b)

In [4]:
n=4

In [5]:
#Calculate distances
def calc_distances(df):
    distances=[]
    for a in df['AQS_Code'].unique():
        lat=list(df[df['AQS_Code']==a]['Latitude'])[0]
        long=list(df[df['AQS_Code']==a]['Longitude'])[0]
        pointa=[lat,long]
        distances_ab=[]
        for b in df['AQS_Code'].unique():
            if b!=a:
                lat=list(df[df['AQS_Code']==b]['Latitude'])[0]
                long=list(df[df['AQS_Code']==b]['Longitude'])[0]
                pointb=[lat,long]
                dist = np.linalg.norm([pointa,pointb])
                distances_ab.append([dist, b])
        distances.append(distances_ab)
    return distances

In [7]:
#Local
df=pd.read_csv('D:/programming-no-gdrive/air-pollution/data-formatted/mpi-houston/Transformed_Data_2000.csv')
#cluster
#df=pd.read_csv('/project/lindner/moving/summer2018/2019/data-formatted/mpi-houston/Transformed_Data_2000.csv')

In [5]:
distances=calc_distances(df)

In [6]:
pickle.dump(distances, open('geo_distances.bin', 'wb'))

In [7]:
aqs=df['AQS_Code'].unique()

In [8]:
distances_dict={}
for index, code in enumerate(aqs):
    distances_dict[code]=distances[index]

In [9]:
pickle.dump(distances_dict, open('geo_distances_dict.bin', 'wb'))

In [10]:
sorted_distances=[]
for site in distances:
    site=sorted(site)
    sorted_distances.append(site)

In [11]:
sorted_distances_dict={}
for index, code in enumerate(aqs):
    sorted_distances_dict[code]=sorted_distances[index]

In [12]:
pickle.dump(sorted_distances, open('geo_distances_sorted.bin', 'wb'))

In [13]:
def weighting(aqs):
    top_n=sorted_distances_dict[aqs][:n]
    total_distance=sum([site[0] for site in top_n])
    weights=[]
    sites=[]
    for site in top_n:
        weight=site[0]/total_distance
        weight=1-weight
        weights.append([weight])
        sites.append(site[1])
    weights=softmax(weights)
    return (weights, sites)

In [14]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [17]:
weights_dict={}
for code in aqs:
    weights=weighting(code)
    weights_dict[code]=weights
weights_dict

{'48_201_1034': (array([[0.25003218],
         [0.24999327],
         [0.24998995],
         [0.2499846 ]]),
  ['48_201_0572', '48_201_0803', '48_201_0551', '48_201_1049']),
 '48_201_0024': (array([[0.25003214],
         [0.24999328],
         [0.24998996],
         [0.24998462]]),
  ['48_201_0572', '48_201_0803', '48_201_0551', '48_201_1049']),
 '48_201_0057': (array([[0.25003218],
         [0.24999327],
         [0.24998995],
         [0.2499846 ]]),
  ['48_201_0572', '48_201_0803', '48_201_0551', '48_201_1049']),
 '48_201_0069': (array([[0.25003217],
         [0.24999328],
         [0.24998995],
         [0.2499846 ]]),
  ['48_201_0572', '48_201_0803', '48_201_0551', '48_201_1049']),
 '48_201_1035': (array([[0.25003217],
         [0.24999328],
         [0.24998995],
         [0.2499846 ]]),
  ['48_201_0572', '48_201_0803', '48_201_0551', '48_201_1049']),
 '48_201_0051': (array([[0.25003212],
         [0.24999329],
         [0.24998997],
         [0.24998463]]),
  ['48_201_0572', '48

In [43]:
#pickle.dump(weights_dict, open('weights_dict.pkl', 'wb'))

In [8]:
weights_dict=pickle.load(open('weights_dict.pkl', 'rb'))

In [9]:
def get_neighbors(pol,sites,epoch,df):
    pols=[]
    for site in sites:
        site_pol=df[(df['epoch']==epoch) & (df['AQS_Code']==site)]
        site_pol=site_pol[pol]
        if site_pol.isnull().all():
            site_pol=np.nan
            pols.append(site_pol)
        else:
            pols.append(site_pol.values.item())
    return pols

In [10]:
def count_and_weigh(pols, thresh, weights):
    counts=collections.Counter(pols)
    if counts[np.nan]>=thresh:
        pol_weighted='insuf'
    else:
        pol_weighted=0.0 
        for index, pol in enumerate(pols):
            pol=weights[index]*pol
            pol_weighted+=pol
    return pol_weighted

In [27]:
def infer_closeby(row, n, thresh):
    #null=row.isnull().any()
    aqs=row['AQS_Code']
    epoch=row['epoch']
    no=np.isnan(row['no'])
    no2=np.isnan(row['no2'])
    nox=np.isnan(row['nox'])
    o3=np.isnan(row['o3'])
    '''if row['no2']==np.nan:
        no2=False
    if row['nox']==np.nan:
        nox=False
    if row['o3']==np.nan:
        o3=False'''
    if no==False or no2==False or nox==False or o3==False:
        profile=weights_dict[aqs]
        sites=profile[1]
        weights=profile[0]
        if no==False:
            no_pols=get_neighbors('no',sites,epoch,df)
            no_weighted=count_and_weigh(no_pols, thresh, weights)
            '''counts=collections.Counter(no_pols)
            if counts[np.nan]>thresh:
                no_pols='insuf'
            else:
                no_pols_weighted=[]
                for index, pol in enumerate(no_pols):
                    pol=weights[index]*pol
                    no_pols_weighted.append(p ol)
                no_pols=no_pols_weighted'''
        else:
            no_weighted=row['no']
        if no2==False:
            no2_pols=get_neighbors('no2', sites,epoch,df)
            no2_weighted=count_and_weigh(no2_pols, thresh, weights)
        else: 
            no2_weighted=row['no2']
        if nox==False:
            nox_pols=get_neighbors('nox', sites,epoch,df)
            nox_weighted=count_and_weigh(nox_pols, thresh, weights)
        else:
            nox_weighted=row['nox']
        if o3==False:
            o3_pols=get_neighbors('o3', sites,epoch,df)
            o3_weighted=count_and_weigh(o3_pols, thresh, weights)
        else:
            o3_weighted=row['o3']
        print([no_weighted,no2_weighted,nox_weighted,o3_weighted])
        return [no_weighted,no2_weighted,nox_weighted,o3_weighted]
    else:
        pass

In [12]:
nulls=df[df.isnull().any(axis=1)]
nulls_no=nulls[nulls['no'].isnull()]

In [22]:
nulls_no=nulls_no.iloc[:100]

In [23]:
epochs=nulls_no['epoch'].unique()
epochs 

array([949518900, 965855700, 965856000, 965856300, 965856600, 965856900,
       965857200, 965857500, 965857800, 965858100, 965858400, 965858700,
       965859000, 965859300, 965859600, 946864800, 946865100, 946865400,
       946865700, 946866000, 946866300, 946866600, 946866900, 946867200,
       946867500, 946867800, 946868100, 946868400, 946868700, 946869000,
       946869300, 946869600, 946869900, 946870200, 946870500, 946870800,
       946871100, 946871400, 946871700, 946872000, 946872300, 946872600,
       946872900, 946873200, 946873500, 946873800, 946874100, 946874400,
       946874700, 946875000, 946875300, 946875600, 946925400, 946925700,
       947010300, 947010600, 947010900, 947011200, 947011500, 947011800,
       947012100, 947012400, 947012700, 947124000, 947124300, 947124600,
       947124900, 947125200, 947125500, 947125800, 947126100, 947126400,
       947126700, 947127000, 947127300, 947127600, 947127900, 947128200,
       947128500, 947128800, 947129100, 947129400, 

In [24]:
len(epochs)

100

In [18]:
nulls_no['epoch'].value_counts().unique()

array([18, 17, 16, 15], dtype=int64)

In [19]:
epoch_splits=np.split(epochs,144) 

In [25]:
epoch_splits=np.split(epochs, 10)

In [42]:
#inferred=nulls_no.apply(lambda x: infer_closeby(x, 4,3), axis=1)

Counter({nan: 3, 29.54112000000001: 1})
Counter({nan: 3, 44.40188: 1})
Counter({nan: 3, 23.02015: 1})
Counter({nan: 3, 5.24212: 1})
Counter({nan: 3, 6.55512: 1})
Counter({nan: 3, 20.659589999999998: 1})
Counter({nan: 3, 9.946819999999999: 1})
Counter({nan: 3, 13.336620000000002: 1})
Counter({nan: 3, 24.4728: 1})
Counter({nan: 3, 17.96032: 1})
Counter({nan: 3, 22.41186: 1})
Counter({nan: 3, 22.29382: 1})
Counter({nan: 3, -6.1318800000000016: 1})
Counter({nan: 3, 417.4841: 1})
Counter({nan: 3, 19.75168: 1})
Counter({nan: 3, 13.410720000000001: 1})
Counter({nan: 3, 168.11441000000005: 1})
Counter({nan: 3, 22.02145: 1})
Counter({nan: 3, -4.68428: 1})
Counter({nan: 3, 74.76903: 1})
Counter({nan: 3, 20.84117: 1})
Counter({nan: 3, -4.5808800000000005: 1})
Counter({nan: 3, 73.52244: 1})
Counter({nan: 3, 22.11224: 1})
Counter({nan: 3, -6.1318800000000016: 1})
Counter({nan: 3, 340.8431700000001: 1})
Counter({nan: 3, 21.83987: 1})
Counter({nan: 3, -6.1318800000000016: 1})
Counter({nan: 3, 420.924

In [28]:
inferred=[]
for split in epoch_splits:
    inferred_split=[]
    for epoch in split:
        print('Epoch '+ str(epoch))
        inferred_slice=nulls_no[nulls_no['epoch']==epoch].apply(lambda x: infer_closeby(x, 4,3), axis=1)
        inferred_split.append(inferred_slice) 
inferred

Epoch 949518900
Epoch 965855700
Epoch 965856000
Epoch 965856300
Epoch 965856600
Epoch 965856900
Epoch 965857200
Epoch 965857500
Epoch 965857800
Epoch 965858100
Epoch 965858400
Epoch 965858700
Epoch 965859000
Epoch 965859300
Epoch 965859600
Epoch 946864800
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946865100
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946865400
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946865700
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946866000
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946866300
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946866600
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946866900
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946867200
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946867500
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946867800
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946868100
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946868400
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946868700
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946869000
[nan, 'insuf', 'insuf', 'insuf']
Epoch 946869300
[nan, 'in

[]

In [47]:
inferred.to_csv('inferred.csv')